In [ ]:
import pandas as pd
import json
import os
from multiprocessing import Pool
from bs4 import BeautifulSoup, SoupStrainer
import itertools
from datetime import datetime

In [70]:
TAKEOUT_PATH = "/home/ivan/Desktop/datasets/Takeout" # CHANGE IF NECESSARY
ACTIVITY_LOG_PATH = os.path.join(TAKEOUT_PATH, "Mi actividad")

ACTIVITY_PLACEHOLDER_NAME = "MiActividad.html"

#drive
FOLDER_DRIVE_ACTIVITY_LOG_PATH = os.path.join(ACTIVITY_LOG_PATH, "Drive")
FILE_DRIVE_ACTIVITY_LOG_PATH = os.path.join(FOLDER_DRIVE_ACTIVITY_LOG_PATH, ACTIVITY_PLACEHOLDER_NAME)

#takeout
FOLDER_DRIVE_ACTIVITY_LOG_PATH = os.path.join(ACTIVITY_LOG_PATH, "Takeout")
FILE_TAKEOUT_ACTIVITY_LOG_PATH = os.path.join(FOLDER_DRIVE_ACTIVITY_LOG_PATH, ACTIVITY_PLACEHOLDER_NAME)

#youtube
FOLDER_DRIVE_ACTIVITY_LOG_PATH = os.path.join(ACTIVITY_LOG_PATH, "YouTube")
FILE_YOUTUBE_ACTIVITY_LOG_PATH = os.path.join(FOLDER_DRIVE_ACTIVITY_LOG_PATH, ACTIVITY_PLACEHOLDER_NAME)

In [77]:
def get_html_contents(file_path, chunk_size=524288):
    """
    Generator function that reads an HTML file in chunks to avoid loading the entire file into memory.
    
    Args:
    - file_path (str): Path to the HTML file.
    - chunk_size (int): Size of each chunk in bytes. Default is 512 KB.
    
    Yields:
    - str: A chunk of HTML that ends on a complete tag.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        buffer = ''
        while True:
            data = file.read(chunk_size)
            if not data:
                # Only yield the remaining buffer if it's a valid chunk with a closed tag
                if buffer:
                    yield buffer
                break

            buffer += data
            last_tag_end = max(buffer.rfind('>'), buffer.rfind('/>'))
            if last_tag_end == -1:
                continue  # Continue reading into buffer until a tag end is found

            # Yield up to the last complete tag and adjust the buffer
            yield buffer[:last_tag_end + 1]
            buffer = buffer[last_tag_end + 1:]


def process_chunk(html_content):
    """
    Processes a chunk of HTML to extract relevant data from specified div elements, including a timestamp.
    
    Args:
    - html_content (str): A string of HTML content.
    
    Returns:
    - list of dict: Extracted data from each content cell in the HTML chunk.
    """
    strainer = SoupStrainer('div', class_="outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp")
    soup = BeautifulSoup(html_content, 'html.parser', parse_only=strainer)
    entries = []
    count = 0
    
    for outer_div in soup.find_all('div', recursive=False):
        content_cells = outer_div.find_all('div', class_="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1")
        for div in content_cells:
            links = div.find_all('a', href=True)
            stripped_strings = list(div.stripped_strings)
            platform = "platform"
            link_action_text = links[0].get_text(strip=True) if len(links) > 0 else ''
            
            if stripped_strings:
                action_parts = stripped_strings[0].split('|')
                action_code = action_parts[0].strip()
                timestamp = stripped_strings[-1].strip()
            else:
                action_code = ''
                timestamp = ''

            entry = {
                "platform": platform,
                "action_code": action_code,
                "timestamp": timestamp,
                "link_action_name": links[0]['href'] if len(links) > 0 else '',
                "link_action_text": link_action_text,
                "channel_link": links[1]['href'] if len(links) > 1 else '',
                "channel_name": links[1].get_text(strip=True) if len(links) > 1 else '',
                "link3": links[2]['href'] if len(links) > 2 else '',
                "link3_text": links[2].get_text(strip=True) if len(links) > 2 else ''
            }
            entries.append(entry)
    
    return entries


def read_html(file_path):
    """
    Main function to process an HTML file, extract data, and write to a CSV file using multiprocessing.
    Dynamically adjusts the number of processes based on the file size.
    
    Args:
    - file_path (str): Path to the HTML file.
    """
    # TO DO: MAKE THE INDIVIDUAL THREADS STORE THE DATA THEMSELVES TO MAKE THE OPERATIONS ASYNC
    file_size = os.path.getsize(file_path)
    chunk_size = 524288  # 512 KB
    num_chunks = (file_size // chunk_size) + 1 
    
    if num_chunks < 4:
        num_processes = num_chunks
    else:
        num_processes = 4
    
    chunks = get_html_contents(file_path, chunk_size=chunk_size)
    with Pool(processes=num_processes) as pool:
        result_iter = pool.imap(process_chunk, chunks)
        results = list(itertools.chain.from_iterable(result_iter))
    
    df = pd.DataFrame(results)
    df.to_csv('output.csv', index=False)
    return df



In [78]:
df = read_html(FILE_DRIVE_ACTIVITY_LOG_PATH)
#print(len(df))
#value_counts = df["action_code"].value_counts().sort_values(ascending=False)    
#value_counts.head(50)

df[df['action_code'].str.len() > 15].head(30)
#value_counts = df["complete_action"].value_counts().sort_values(ascending=False)  
#value_counts.head(50) 


,platform,action_code,timestamp,link_action_name,link_action_text,channel_link,channel_name,link3,link3_text
0,platform,Se ha iniciado una descarga en bloque en Drive,"10 sept 2023, 18:29:17 CET",,,,,,
1,platform,Se ha iniciado una descarga en bloque en Drive,"3 dic 2021, 19:36:31 CET",,,,,,
